<a href="https://colab.research.google.com/github/HitsujiAura/AbstractArtGAN/blob/master/stylegan2-ada-pytorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Custom Training StyleGan2-ADA

StyleGAN2-ADA only work with Tensorflow 1. Run the next cell before anything else to make sure we’re using TF1 and not TF2.

In [1]:
%tensorflow_version 1.x

TensorFlow 1.x selected.


In [2]:
!nvidia-smi

Thu May  6 20:11:11 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 465.19.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   66C    P8    12W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

## Install Repo to Google Drive

Colab is a little funky with training. I’ve found the best way to do this is to install the repo directly into your Google Drive folder.

First, mount your Drive to the Colab notebook: 

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Next, run this cell. If you’re already installed the repo, it will skip the installation process and change into the repo’s directory. If you haven’t installed it, it will install all the files necessary.

In [11]:
import os
if os.path.isdir("/content/drive/My Drive/colab-sg2-ada"):
    %cd "/content/drive/My Drive/colab-sg2-ada/stylegan2-ada-pytorch"
else:
    #install script
    %cd "/content/drive/My Drive/"
    !mkdir colab-sg2-ada
    %cd colab-sg2-ada
    !git clone https://github.com/NVlabs/stylegan2-ada-pytorch.git
    %cd stylegan2-ada-pytorch
    !mkdir downloads
    !mkdir datasets

/content/drive/My Drive
/content/drive/My Drive/colab-sg2-ada
Cloning into 'stylegan2-ada-pytorch'...
remote: Enumerating objects: 125, done.
remote: Total 125 (delta 0), reused 0 (delta 0), pack-reused 125
Receiving objects: 100% (125/125), 1.12 MiB | 2.76 MiB/s, done.
Resolving deltas: 100% (55/55), done.
/content/drive/My Drive/colab-sg2-ada/stylegan2-ada-pytorch


In [ ]:
# %cd "/content/drive/My Drive/colab-sg2-ada/stylegan2-ada"
# !git config --global user.name "test"
# !git config --global user.email "test@test.com"
# !git fetch origin
# !git checkout origin/main -- train.py

## Convert dataset to .tfrecords

**Note: You only need to do this once per dataset. If you have already run this and are returning to conntinue training, skip these cells.**

Next we need to convert our image dataset to a format that StyleGAN2-ADA can read from. There are two options here. You can upload your dataset directly to Colab (as a zipped file), or you can upload it to Drive directly and read it from there.

In [5]:
#if you manually uploaded your dataset to Colab, unzip it
zip_path = "/content/drive/MyDrive/Datasets/abstract-art.zip"

!unzip {zip_path} -d /content/

Streaming output truncated to the last 5000 lines.
  inflating: /content/abstract_art_512/abstract_huguette-arthur-bertrand_1492.jpg  
  inflating: /content/abstract_art_512/abstract_huguette-arthur-bertrand_1493.jpg  
  inflating: /content/abstract_art_512/abstract_huguette-arthur-bertrand_1494.jpg  
  inflating: /content/abstract_art_512/abstract_huguette-arthur-bertrand_1495.jpg  
  inflating: /content/abstract_art_512/abstract_huguette-arthur-bertrand_1496.jpg  
  inflating: /content/abstract_art_512/abstract_huguette-arthur-bertrand_1497.jpg  
  inflating: /content/abstract_art_512/abstract_huguette-arthur-bertrand_1498.jpg  
  inflating: /content/abstract_art_512/abstract_huguette-arthur-bertrand_1499.jpg  
  inflating: /content/abstract_art_512/abstract_huguette-arthur-bertrand_1500.jpg  
  inflating: /content/abstract_art_512/abstract_huguette-arthur-bertrand_1501.jpg  
  inflating: /content/abstract_art_512/abstract_iain-baxter_1294.jpg  
  inflating: /content/abstract_art_512

Now that your image dataset is uploaded, we need to convert it to the `.tfrecords` format.

Depending on the resolution of your images and how many you have, this can take a while.

In [13]:
#update this to the path to your image folder
dataset_path = "/content/abstract_art_512"
#give your dataset a name
dataset_name = "abstract_art_512"

#you don't need to edit anything here
!python dataset_tool.py --source=/content/abstract_art_512/ --dest=./datasets/abstract_art_512.zip

100% 8145/8145 [02:58<00:00, 45.60it/s]


## Train a custom model

We’re ready to start training! There are numerous arguments to training, what’s listed below are the most popular options. To see all the options, run the following cell.

In [14]:
!python train.py --help

Usage: train.py [OPTIONS]

  Train a GAN using the techniques described in
  the paper "Training Generative Adversarial
  Networks with Limited Data".

  Examples:

  # Train with custom dataset using 1 GPU.
  python train.py --outdir=~/training-runs --data=~/mydataset.zip --gpus=1

  # Train class-conditional CIFAR-10 using 2 GPUs.
  python train.py --outdir=~/training-runs --data=~/datasets/cifar10.zip \
      --gpus=2 --cfg=cifar --cond=1

  # Transfer learn MetFaces from FFHQ using 4 GPUs.
  python train.py --outdir=~/training-runs --data=~/datasets/metfaces.zip \
      --gpus=4 --cfg=paper1024 --mirror=1 --resume=ffhq1024 --snap=10

  # Reproduce original StyleGAN2 config F.
  python train.py --outdir=~/training-runs --data=~/datasets/ffhq.zip \
      --gpus=8 --cfg=stylegan2 --mirror=1 --aug=noaug

  Base configs (--cfg):
    auto       Automatically select reasonable defaults based on resolution
               and GPU count. Good starting point for new datasets.
    stylegan2  R

In [ ]:
#this name must EXACTLY match the dataset name you used when creating the .tfrecords file
dataset_name = "abstract_art_512"
#how often should the model generate samples and a .pkl file
snapshot_count = 2
#should the images be mirrored left to right?
mirrored = False
#should the images be mirrored top to bottom?
mirroredY = False
#metrics? 
metric_list = None
#augments
augs = "bgc"

#
# this is the most important cell to update
#
# running it for the first time? set it to ffhq(+resolution)
# resuming? get the path to your latest .pkl file and use that
# resume_from = "/content/drive/My\ Drive/colab-sg2-ada2/stylegan2-ada/results/00008-dante1024-mirror-mirrory-11gb-gpu-bg-resumecustom/network-snapshot-000160.pkl"
resume_from = "ffhq512"

#don't edit this unless you know what you're doing :)
!python train.py --outdir ./results --snap={snapshot_count} --cfg=11gb-gpu --data=./datasets/{dataset_name} --augpipe={augs} --mirror={mirrored} --mirrory={mirroredY} --metrics={metric_list} --resume={resume_from}

In [ ]:
!python train.py --outdir=./results --data=./datasets/abstract_art_512.zip --gpus=1

Streaming output truncated to the last 5000 lines.
/content/drive/My Drive/colab-sg2-ada/stylegan2-ada-pytorch/torch_utils/ops/upfirdn2d.py:34: UserWarning: Failed to build CUDA kernels for upfirdn2d. Falling back to slow reference implementation. Details:

Traceback (most recent call last):
  File "/content/drive/My Drive/colab-sg2-ada/stylegan2-ada-pytorch/torch_utils/ops/upfirdn2d.py", line 32, in _init
    _plugin = custom_ops.get_plugin('upfirdn2d_plugin', sources=sources, extra_cuda_cflags=['--use_fast_math'])
  File "/content/drive/My Drive/colab-sg2-ada/stylegan2-ada-pytorch/torch_utils/custom_ops.py", line 110, in get_plugin
    torch.utils.cpp_extension.load(name=module_name, verbose=verbose_build, sources=sources, **build_kwargs)
  File "/usr/local/lib/python3.7/dist-packages/torch/utils/cpp_extension.py", line 1091, in load
    keep_intermediates=keep_intermediates)
  File "/usr/local/lib/python3.7/dist-packages/torch/utils/cpp_extension.py", line 1317, in _jit_compile
    

### While it’s training...
**Once the above cell is running you should be training!**

Don’t close this tab! Colab needs to be open and running in order to continue training. Every ~15min or so a new line should get added to your output, indicated its still training. Depending on you `snapshot_count` setting you should see the results folder in your Google drive folder fill with both samples (`fakesXXXXXx.jpg`) and model weights (`network-snapshot-XXXXXX.pkl`). The samples are worth looking at while it trains but don’t get too worried about each individual sample.

If you chose a metric, you will also see scores for each snapshot. Don’t obsess over these! they are a guide, it can go up or down slightly for each snapshot. What you want to see is a gradual lowering of the score over time.

Once Colab shuts off, you can Reconnect the notebook and re-run every cell from top to bottom. Make sure you update the `resume_from` path to continue training from the latest model.